In [ ]:
# default_exp utils.common_utils

# Common utils
> A collection of utilities often used.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import sys
import os
import ssl
import os.path as osp
from six.moves import urllib
import errno
import tarfile
import zipfile
import bz2
import gzip
import pandas as pd
from tabulate import tabulate

## Download

In [ ]:
#exporti
def makedirs(path):
    try:
        os.makedirs(osp.expanduser(osp.normpath(path)))
    except OSError as e:
        if e.errno != errno.EEXIST and osp.isdir(path):
            raise e

In [ ]:
#export
def wget_download(url, savepath):
    import wget
    wget.download(url, str(savepath))

In [ ]:
#export
def download_url(url: str, folder: str, log: bool = True):
    r"""Downloads the content of an URL to a specific folder.
    Args:
        url (string): The url.
        folder (string): The folder.
        log (bool, optional): If :obj:`False`, will not print anything to the
            console. (default: :obj:`True`)
    """

    filename = url.rpartition('/')[2]
    filename = filename if filename[0] == '?' else filename.split('?')[0]
    path = osp.join(folder, filename)

    if osp.exists(path):  # pragma: no cover
        if log:
            print(f'Using existing file {filename}', file=sys.stderr)
        return path

    if log:
        print(f'Downloading {url}', file=sys.stderr)

    makedirs(folder)

    context = ssl._create_unverified_context()
    data = urllib.request.urlopen(url, context=context)

    with open(path, 'wb') as f:
        f.write(data.read())

    return path

In [ ]:
download_url('https://files.grouplens.org/datasets/movielens/ml-1m.zip',
             './data/bronze')

'./data/bronze/ml-1m.zip'

In [ ]:
!tree ./data

./data
└── bronze
    └── ml-1m.zip

1 directory, 1 file


In [ ]:
!tree --du -h -C ./data

./data
├── [ 24M]  bronze
│   └── [ 24M]  ml-1m
│       ├── [167K]  movies.dat
│       ├── [ 23M]  ratings.dat
│       ├── [5.4K]  README
│       └── [131K]  users.dat
└── [3.0M]  silver
    └── [3.0M]  ml-1m_min_rating0-min_uc5-min_sc5-splitleave_one_out
        └── [3.0M]  dataset.pkl

  27M used in 4 directories, 5 files


---

## Extract

In [ ]:
#exporti
def maybe_log(path, log=True):
    if log:
        print(f'Extracting {path}', file=sys.stderr)

In [ ]:
#export
def extract_tar(path: str, folder: str, mode: str = 'r:gz', log: bool = True):
    r"""Extracts a tar archive to a specific folder.
    Args:
        path (string): The path to the tar archive.
        folder (string): The folder.
        mode (string, optional): The compression mode. (default: :obj:`"r:gz"`)
        log (bool, optional): If :obj:`False`, will not print anything to the
            console. (default: :obj:`True`)
    """
    maybe_log(path, log)
    with tarfile.open(path, mode) as f:
        f.extractall(folder)

In [ ]:
#export
def extract_zip(path: str, folder: str, log: bool = True):
    r"""Extracts a zip archive to a specific folder.
    Args:
        path (string): The path to the tar archive.
        folder (string): The folder.
        log (bool, optional): If :obj:`False`, will not print anything to the
            console. (default: :obj:`True`)
    """
    maybe_log(path, log)
    with zipfile.ZipFile(path, 'r') as f:
        f.extractall(folder)

In [ ]:
#export
def extract_bz2(path: str, folder: str, log: bool = True):
    r"""Extracts a bz2 archive to a specific folder.
    Args:
        path (string): The path to the tar archive.
        folder (string): The folder.
        log (bool, optional): If :obj:`False`, will not print anything to the
            console. (default: :obj:`True`)
    """
    maybe_log(path, log)
    path = osp.abspath(path)
    with bz2.open(path, 'r') as r:
        with open(osp.join(folder, '.'.join(path.split('.')[:-1])), 'wb') as w:
            w.write(r.read())

In [ ]:
#export
def extract_gz(path: str, folder: str, log: bool = True):
    r"""Extracts a gz archive to a specific folder.
    Args:
        path (string): The path to the tar archive.
        folder (string): The folder.
        log (bool, optional): If :obj:`False`, will not print anything to the
            console. (default: :obj:`True`)
    """
    maybe_log(path, log)
    path = osp.abspath(path)
    with gzip.open(path, 'r') as r:
        with open(osp.join(folder, '.'.join(path.split('.')[:-1])), 'wb') as w:
            w.write(r.read())

## Tabulate

In [ ]:
#export
def print_result_as_table(results, tag=None):
    """Print results as a table."""
    eval_infos = set()
    for result in results:
        eval_infos.update(result.keys())
    eval_infos = list(eval_infos)
    print("-" * 80)
    if tag is not None:
        print(tag)
    for result in results:
        for eval_info in eval_infos:
            if eval_info not in result:
                result[eval_info] = "--"
    df = pd.DataFrame(results)
    df = df.set_index("model")
    df = df.T
    print(tabulate(df, headers=df.columns, tablefmt="psql"))
    print("-" * 80)

In [ ]:
results = [{'model':'MF', 'MRR':.35},
           {'model':'NCF', 'MRR':.42, 'nDCG':.25}]

print_result_as_table(results)

--------------------------------------------------------------------------------
+------+------+-------+
|      | MF   |   NCF |
|------+------+-------|
| MRR  | 0.35 |  0.42 |
| nDCG | --   |  0.25 |
+------+------+-------+
--------------------------------------------------------------------------------


## Listing

In [ ]:
#export
def list_files(startpath):
    """
    Util function to print the nested structure of a directory
    """
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, "").count(os.sep)
        indent = " " * 4 * (level)
        print("{}{}/".format(indent, os.path.basename(root)))
        subindent = " " * 4 * (level + 1)
        for f in files:
            print("{}{}".format(subindent, f))

In [ ]:
list_files('./sample_data')

sample_data/
    README.md
    anscombe.json
    california_housing_train.csv
    california_housing_test.csv
    mnist_test.csv
    mnist_train_small.csv


In [ ]:
#hide
%reload_ext watermark
%watermark -a "Sparsh A." -m -iv -u -t -d

Author: Sparsh A.

Last updated: 2021-12-21 12:52:30

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.4.104+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit

sys    : 3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]
google : 2.0.3
numpy  : 1.19.5
pandas : 1.1.5
tarfile: 0.9.0
six    : 1.15.0
IPython: 5.5.0

